In [ ]:
import datetime
import hashlib
import itertools
import json
import os
import pickle
import re
import warnings

from collections import namedtuple
from copy import copy
from pathlib import Path

In [ ]:
import cartopy.feature as cfeature
import cartopy.crs as ccrs

import joblib

import matplotlib as mpl
from matplotlib import gridspec
from matplotlib import pyplot as plt

import netCDF4
from netCDF4 import Dataset

import numpy as np

import pandas as pd

import scipy as sp

import sklearn
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

import tqdm

In [ ]:
import colorcet as cc

uncertainty_cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'uncertainty_cmap', cc.m_CET_L8(np.linspace(0, 0.9, 1000)),
)
density_cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'density_cmap', np.vstack([
        [(1.0, 1.0, 1.0, 1.0)], cc.m_CET_L8(np.linspace(0, 0.9, 1000)),
    ])
)

In [ ]:
# https://stackoverflow.com/a/49601444
def adjust_lightness(color, amount=0.5):
    import matplotlib.colors as mc
    import colorsys

    try:
        c = mc.cnames[color]
    except:
        c = color

    c = colorsys.rgb_to_hls(*mc.to_rgb(c))

    return colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2])

def replace_lightness(color, lightness):
    import matplotlib.colors as mc
    import colorsys

    try:
        c = mc.cnames[color]
    except:
        c = color

    c = colorsys.rgb_to_hls(*mc.to_rgb(c))

    return colorsys.hls_to_rgb(c[0], max(0, min(1, lightness)), c[2])

In [ ]:
TrajectoryPaths = namedtuple("TrajectoryPaths", ["date", "out", "aer", "ant", "bio", "met"])
TrajectoryDatasets = namedtuple("TrajectoryDatasets", ["date", "out", "aer", "ant", "bio", "met"])
MLDataset = namedtuple("MLDataset", ["date", "paths", "X_raw", "Y_raw", "X_train", "X_valid", "X_test", "Y_train", "Y_valid", "Y_test", "X_scaler", "Y_scaler"])

In [ ]:
OUTDIR_PATTERN = re.compile(r"(\d{4})(\d{2})(\d{2})_T(\d{2})")

In [ ]:
traj_datetimes = dict()

base = Path.cwd().parent / "trajectories"

for child in (base / "outputs" / "baseline").iterdir():
    if not child.is_dir():
        continue
    
    match = OUTDIR_PATTERN.match(child.name)
    
    if match is None:
        continue
        
    date = datetime.datetime(
        year=int(match.group(1)),
        month=int(match.group(2)),
        day=int(match.group(3)),
        hour=int(match.group(4)),
    )
    
    out_path = child / "output.nc"
    aer_path = (
        base / "inputs" / "baseline" / "HYDE_BASE_Y2018" /
        f"OUTPUT_bwd_{date.strftime('%Y%m%d')}" /
        "EMISSIONS_0422" /
        f"{date.strftime('%Y%m%d')}_7daybwd_Hyde_traj_AER_{24-date.hour:02}_L3.nc"
    )
    ant_path = (
        base / "inputs" / "baseline" / "HYDE_BASE_Y2018" /
        f"OUTPUT_bwd_{date.strftime('%Y%m%d')}" /
        "EMISSIONS_0422" /
        f"{date.strftime('%Y%m%d')}_7daybwd_Hyde_traj_ANT_{24-date.hour:02}_L3.nc"
    )
    bio_path = (
        base / "inputs" / "baseline" / "HYDE_BASE_Y2018" /
        f"OUTPUT_bwd_{date.strftime('%Y%m%d')}" /
        "EMISSIONS_0422" /
        f"{date.strftime('%Y%m%d')}_7daybwd_Hyde_traj_BIO_{24-date.hour:02}_L3.nc"
    )
    met_path = (
        base / "inputs" / "baseline" / "HYDE_BASE_Y2018" /
        f"OUTPUT_bwd_{date.strftime('%Y%m%d')}" /
        "METEO" /
        f"METEO_{date.strftime('%Y%m%d')}_R{24-date.hour:02}.nc"
    )
    
    if (
        (not out_path.exists()) or (not aer_path.exists()) or
        (not ant_path.exists()) or (not bio_path.exists()) or
        (not met_path.exists())
    ):
        raise Exception(out_path, aer_path, ant_path, bio_path, met_path)
    
    traj_datetimes[date] = TrajectoryPaths(
        date=date, out=out_path, aer=aer_path, ant=ant_path, bio=bio_path, met=met_path,
    )

traj_dates = sorted(set(d.date() for d in traj_datetimes.keys()))

In [ ]:
def load_trajectory_dataset(paths: TrajectoryPaths) -> TrajectoryDatasets:
    outds = Dataset(paths.out, "r", format="NETCDF4")
    aerds = Dataset(paths.aer, "r", format="NETCDF4")
    antds = Dataset(paths.ant, "r", format="NETCDF4")
    biods = Dataset(paths.bio, "r", format="NETCDF4")
    metds = Dataset(paths.met, "r", format="NETCDF4")
    
    return TrajectoryDatasets(
        date=paths.date, out=outds, aer=aerds, ant=antds, bio=biods, met=metds,
    )

In [ ]:
data_proj = ccrs.PlateCarree()
projection = ccrs.LambertConformal(
    central_latitude=50, central_longitude=20, standard_parallels=(25, 25)
)
extent = [-60, 60, 40, 80]

In [ ]:
def get_ccn_concentration(ds: TrajectoryDatasets):
    ccn_bin_indices, = np.nonzero(ds.out["dp_dry_fs"][:].data > 80e-9)
    ccn_concentration = np.sum(ds.out["nconc_par"][:].data[:,ccn_bin_indices,:], axis=1)
    
    return pd.DataFrame({
        "time": np.repeat(get_output_time(ds), ds.out["lev"].shape[0]),
        "level": np.tile(ds.out["lev"][:].data, ds.out["time"].shape[0]),
        "ccn": ccn_concentration.flatten(),
    }).set_index(["time", "level"])

In [ ]:
def get_output_time(ds: TrajectoryDatasets):
    fdom = datetime.datetime.strptime(
        ds.out["time"].__dict__["first_day_of_month"], "%Y-%m-%d %H:%M:%S",
    )
    dt = (ds.date - fdom).total_seconds()
    
    out_t = ds.out["time"][:].data
    
    return out_t - dt

def interpolate_meteorology_values(ds: TrajectoryDatasets, key: str):
    out_t = get_output_time(ds)
    out_h = ds.out["lev"][:].data
    
    met_t = ds.met["time"][:].data
    met_h = ds.met["lev"][:].data
    
    met_t_h = ds.met[key][:]
    
    met_t_h_int = sp.interpolate.interp2d(
        x=met_h, y=met_t, z=met_t_h, kind="linear", bounds_error=False, fill_value=0.0,
    )
    
    return met_t_h_int(x=out_h, y=out_t)

def interpolate_meteorology_time_values(ds: TrajectoryDatasets, key: str):
    out_t = get_output_time(ds)
    out_h = ds.out["lev"][:].data
    
    met_t = ds.met["time"][:].data
    
    met_t_v = ds.met[key][:]
    
    met_t_int = sp.interpolate.interp1d(
        x=met_t, y=met_t_v, kind="linear", bounds_error=False, fill_value=0.0,
    )
    
    return np.repeat(
        met_t_int(x=out_t).reshape(-1, 1),
        out_h.shape[0], axis=1,
    )

def interpolate_biogenic_emissions(ds: TrajectoryDatasets, key: str):
    out_t = get_output_time(ds)
    out_h = ds.out["lev"][:].data
    
    # depth of each box layer, assuming level heights are midpoints and end points are clamped
    out_d = (np.array(list(out_h[1:])+[out_h[-1]]) - np.array([out_h[0]]+list(out_h[:-1]))) / 2.0
    
    bio_t = ds.bio["time"][:].data
    
    # Biogenic emissions are limited to boxes at <= 10m height
    biogenic_emission_layers = np.nonzero(out_h <= 10.0)
    biogenic_emission_layer_height_cumsum = np.cumsum(out_d[biogenic_emission_layers])
    biogenic_emission_layer_proportion = biogenic_emission_layer_height_cumsum / biogenic_emission_layer_height_cumsum[-1]
    num_biogenic_emission_layers = sum(out_h <= 10.0)
    
    bio_t_h = np.zeros(shape=(out_t.size, out_h.size))
    
    bio_t_int = sp.interpolate.interp1d(
        x=bio_t, y=ds.bio[key][:], kind="linear", bounds_error=False, fill_value=0.0,
    )
    
    # Split up the biogenic emissions relative to the depth of the boxes
    bio_t_h[:,biogenic_emission_layers] = (
        np.tile(bio_t_int(x=out_t), (num_biogenic_emission_layers, 1, 1)) * biogenic_emission_layer_proportion.reshape(-1, 1, 1)
    ).T
    
    return bio_t_h

def interpolate_aerosol_emissions(ds: TrajectoryDatasets, key: str):
    out_t = get_output_time(ds)
    out_h = ds.out["lev"][:].data
    
    aer_t = ds.aer["time"][:].data
    aer_h = ds.aer["mid_layer_height"][:].data
    
    aer_t_h = ds.aer[key][:].T
    
    aer_t_h_int = sp.interpolate.interp2d(
        x=aer_h, y=aer_t, z=aer_t_h, kind="linear", bounds_error=False, fill_value=0.0,
    )
    
    return aer_t_h_int(x=out_h, y=out_t)

def interpolate_anthropogenic_emissions(ds: TrajectoryDatasets, key: str):
    out_t = get_output_time(ds)
    out_h = ds.out["lev"][:].data
    
    ant_t = ds.ant["time"][:].data
    ant_h = ds.ant["mid_layer_height"][:].data
    
    ant_t_h = ds.ant[key][:].T
    
    ant_t_h_int = sp.interpolate.interp2d(
        x=ant_h, y=ant_t, z=ant_t_h, kind="linear", bounds_error=False, fill_value=0.0,
    )
    
    return ant_t_h_int(x=out_h, y=out_t)

In [ ]:
def get_meteorology_features(ds: TrajectoryDatasets):
    return pd.DataFrame({
        "time": np.repeat(get_output_time(ds), ds.out["lev"].shape[0]),
        "level": np.tile(ds.out["lev"][:].data, ds.out["time"].shape[0]),
        "met_t": interpolate_meteorology_values(ds, "t").flatten(),
        # "met_u": interpolate_meteorology_values(ds, "u").flatten(),
        # "met_v": interpolate_meteorology_values(ds, "v").flatten(),
        "met_q": interpolate_meteorology_values(ds, "q").flatten(),
        # "met_qc": interpolate_meteorology_values(ds, "qc").flatten(),
        # "met_sp": interpolate_meteorology_time_values(ds, "sp").flatten(),
        # "met_cp": interpolate_meteorology_time_values(ds, "cp").flatten(),
        # "met_sshf": interpolate_meteorology_time_values(ds, "sshf").flatten(),
        "met_ssr": interpolate_meteorology_time_values(ds, "ssr").flatten(),
        # "met_lsp": interpolate_meteorology_time_values(ds, "lsp").flatten(),
        # "met_ewss": interpolate_meteorology_time_values(ds, "ewss").flatten(),
        # "met_nsss": interpolate_meteorology_time_values(ds, "nsss").flatten(),
        # "met_tcc": interpolate_meteorology_time_values(ds, "tcc").flatten(),
        "met_lsm": interpolate_meteorology_time_values(ds, "lsm").flatten(),
        # "met_omega": interpolate_meteorology_values(ds, "omega").flatten(),
        # "met_z": interpolate_meteorology_time_values(ds, "z").flatten(),
        # "met_mla": interpolate_meteorology_values(ds, "mla").flatten(),
        # NOTE: lp is excluded because it allows the model to overfit
        # "met_lp": interpolate_meteorology_values(ds, "lp").flatten(),
        "met_blh": interpolate_meteorology_time_values(ds, "blh").flatten(),
    }).set_index(["time", "level"])

def get_bio_emissions_features(ds: TrajectoryDatasets):
    return pd.DataFrame({
        "time": np.repeat(get_output_time(ds), ds.out["lev"].shape[0]),
        "level": np.tile(ds.out["lev"][:].data, ds.out["time"].shape[0]),
        "bio_acetaldehyde": interpolate_biogenic_emissions(ds, "acetaldehyde").flatten(),
        "bio_acetone": interpolate_biogenic_emissions(ds, "acetone").flatten(),
        "bio_butanes_and_higher_alkanes": interpolate_biogenic_emissions(ds, "butanes-and-higher-alkanes").flatten(),
        "bio_butanes_and_higher_alkenes": interpolate_biogenic_emissions(ds, "butenes-and-higher-alkenes").flatten(),
        "bio_ch4": interpolate_biogenic_emissions(ds, "CH4").flatten(),
        "bio_co": interpolate_biogenic_emissions(ds, "CO").flatten(),
        "bio_ethane": interpolate_biogenic_emissions(ds, "ethane").flatten(),
        "bio_ethanol": interpolate_biogenic_emissions(ds, "ethanol").flatten(),
        "bio_ethene": interpolate_biogenic_emissions(ds, "ethene").flatten(),
        "bio_formaldehyde": interpolate_biogenic_emissions(ds, "formaldehyde").flatten(),
        "bio_hydrogen_cyanide": interpolate_biogenic_emissions(ds, "hydrogen-cyanide").flatten(),
        "bio_iosprene": interpolate_biogenic_emissions(ds, "isoprene").flatten(),
        "bio_mbo": interpolate_biogenic_emissions(ds, "MBO").flatten(),
        "bio_methanol": interpolate_biogenic_emissions(ds, "methanol").flatten(),
        "bio_methyl_bromide": interpolate_biogenic_emissions(ds, "methyl-bromide").flatten(),
        "bio_methyl_chloride": interpolate_biogenic_emissions(ds, "methyl-chloride").flatten(),
        "bio_methyl_iodide": interpolate_biogenic_emissions(ds, "methyl-iodide").flatten(),
        "bio_other_aldehydes": interpolate_biogenic_emissions(ds, "other-aldehydes").flatten(),
        "bio_other_ketones": interpolate_biogenic_emissions(ds, "other-ketones").flatten(),
        "bio_other_monoterpenes": interpolate_biogenic_emissions(ds, "other-monoterpenes").flatten(),
        "bio_pinene_a": interpolate_biogenic_emissions(ds, "pinene-a").flatten(),
        "bio_pinene_b": interpolate_biogenic_emissions(ds, "pinene-b").flatten(),
        "bio_propane": interpolate_biogenic_emissions(ds, "propane").flatten(),
        "bio_propene": interpolate_biogenic_emissions(ds, "propene").flatten(),
        "bio_sesquiterpenes": interpolate_biogenic_emissions(ds, "sesquiterpenes").flatten(),
        "bio_toluene": interpolate_biogenic_emissions(ds, "toluene").flatten(),
        "bio_ch2br2": interpolate_biogenic_emissions(ds, "CH2Br2").flatten(),
        "bio_ch3i": interpolate_biogenic_emissions(ds, "CH3I").flatten(),
        "bio_chbr3": interpolate_biogenic_emissions(ds, "CHBr3").flatten(),
        "bio_dms": interpolate_biogenic_emissions(ds, "DMS").flatten(),
    }).set_index(["time", "level"])

def get_aer_emissions_features(ds: TrajectoryDatasets):
    return pd.DataFrame({
        "time": np.repeat(get_output_time(ds), ds.out["lev"].shape[0]),
        "level": np.tile(ds.out["lev"][:].data, ds.out["time"].shape[0]),
        "aer_3_10_nm": interpolate_aerosol_emissions(ds, "3-10nm").flatten(),
        "aer_10_20_nm": interpolate_aerosol_emissions(ds, "10-20nm").flatten(),
        "aer_20_30_nm": interpolate_aerosol_emissions(ds, "20-30nm").flatten(),
        "aer_30_50_nm": interpolate_aerosol_emissions(ds, "30-50nm").flatten(),
        "aer_50_70_nm": interpolate_aerosol_emissions(ds, "50-70nm").flatten(),
        "aer_70_100_nm": interpolate_aerosol_emissions(ds, "70-100nm").flatten(),
        "aer_100_200_nm": interpolate_aerosol_emissions(ds, "100-200nm").flatten(),
        "aer_200_400_nm": interpolate_aerosol_emissions(ds, "200-400nm").flatten(),
        "aer_400_1000_nm": interpolate_aerosol_emissions(ds, "400-1000nm").flatten(),
    }).set_index(["time", "level"])

def get_ant_emissions_features(ds: TrajectoryDatasets):
    return pd.DataFrame({
        "time": np.repeat(get_output_time(ds), ds.out["lev"].shape[0]),
        "level": np.tile(ds.out["lev"][:].data, ds.out["time"].shape[0]),
        "ant_co": interpolate_anthropogenic_emissions(ds, "co").flatten(),
        "ant_nox": interpolate_anthropogenic_emissions(ds, "nox").flatten(),
        "ant_co2": interpolate_anthropogenic_emissions(ds, "co2").flatten(),
        "ant_nh3": interpolate_anthropogenic_emissions(ds, "nh3").flatten(),
        "ant_ch4": interpolate_anthropogenic_emissions(ds, "ch4").flatten(),
        "ant_so2": interpolate_anthropogenic_emissions(ds, "so2").flatten(),
        "ant_nmvoc": interpolate_anthropogenic_emissions(ds, "nmvoc").flatten(),
        "ant_alcohols": interpolate_anthropogenic_emissions(ds, "alcohols").flatten(),
        "ant_ethane": interpolate_anthropogenic_emissions(ds, "ethane").flatten(),
        "ant_propane": interpolate_anthropogenic_emissions(ds, "propane").flatten(),
        "ant_butanes": interpolate_anthropogenic_emissions(ds, "butanes").flatten(),
        "ant_pentanes": interpolate_anthropogenic_emissions(ds, "pentanes").flatten(),
        "ant_hexanes": interpolate_anthropogenic_emissions(ds, "hexanes").flatten(),
        "ant_ethene": interpolate_anthropogenic_emissions(ds, "ethene").flatten(),
        "ant_propene": interpolate_anthropogenic_emissions(ds, "propene").flatten(),
        "ant_acetylene": interpolate_anthropogenic_emissions(ds, "acetylene").flatten(),
        "ant_isoprene": interpolate_anthropogenic_emissions(ds, "isoprene").flatten(),
        "ant_monoterpenes": interpolate_anthropogenic_emissions(ds, "monoterpenes").flatten(),
        "ant_other_alkenes_and_alkynes": interpolate_anthropogenic_emissions(ds, "other-alkenes-and-alkynes").flatten(),
        "ant_benzene": interpolate_anthropogenic_emissions(ds, "benzene").flatten(),
        "ant_toluene": interpolate_anthropogenic_emissions(ds, "toluene").flatten(),
        "ant_xylene": interpolate_anthropogenic_emissions(ds, "xylene").flatten(),
        "ant_trimethylbenzene": interpolate_anthropogenic_emissions(ds, "trimethylbenzene").flatten(),
        "ant_other_aromatics": interpolate_anthropogenic_emissions(ds, "other-aromatics").flatten(),
        "ant_esters": interpolate_anthropogenic_emissions(ds, "esters").flatten(),
        "ant_ethers": interpolate_anthropogenic_emissions(ds, "ethers").flatten(),
        "ant_formaldehyde": interpolate_anthropogenic_emissions(ds, "formaldehyde").flatten(),
        "ant_other_aldehydes": interpolate_anthropogenic_emissions(ds, "other-aldehydes").flatten(),
        "ant_total_ketones": interpolate_anthropogenic_emissions(ds, "total-ketones").flatten(),
        "ant_total_acids": interpolate_anthropogenic_emissions(ds, "total-acids").flatten(),
        "ant_other_vocs": interpolate_anthropogenic_emissions(ds, "other-VOCs").flatten(),
    }).set_index(["time", "level"])

In [ ]:
# https://stackoverflow.com/a/67809235
def df_to_numpy(df):
    try:
        shape = [len(level) for level in df.index.levels]
    except AttributeError:
        shape = [len(df.index)]
    ncol = df.shape[-1]
    if ncol > 1:
        shape.append(ncol)
    return df.to_numpy().reshape(shape)

In [ ]:
def generate_time_level_windows():
    # -0.5h, -1.5h, -3h, -6h, -12h, -24h, -48h
    # 0, -2, -5, -11, -23, -47, -95
    time_windows = [(0, 0), (-2, -1), (-5, -3), (-11, -6), (-23, -12), (-47, -24), (-95, -48)]
    
    # +1l, +2l, +4l, +8l, +16l, +32l, +64
    top_windows = [(1, 1), (1, 2), (1, 4), (2, 8), (2, 16), (3, 32), (3, 64)]
    mid_windows = [(0, 0), (0, 0), (0, 0), (-1, 1), (-1, 1), (-2, 2), (-2, 2)]
    bot_windows = [(-1, -1), (-2, -1), (-4, -1), (-8, -2), (-16, -2), (-32, -3), (-64, -3)]
    
    return list(itertools.chain(
        zip(time_windows, top_windows), zip(time_windows, mid_windows), zip(time_windows, bot_windows),
    ))

In [ ]:
def generate_windowed_feature_names(columns):
    time_windows = ["-0.5h", "-1.5h", "-3h", "-6h", "-12h", "-24h", "-48h"]
    
    top_windows = ["+1l", "+2l", "+4l", "+8l", "+16l", "+32l", "+64l"]
    mid_windows = ["+0l", "+0l", "+0l", "±1l", "±1l", "±2l", "±2l"]
    bot_windows = ["-1l", "-2l", "-4l", "-8l", "-16l", "-32l", "-64l"]
    
    names = []
    
    for (t, l) in itertools.chain(
        zip(time_windows, top_windows), zip(time_windows, mid_windows), zip(time_windows, bot_windows),
    ):
        for c in columns:
            names.append(f"{c}{t}{l}")
    
    return names

In [ ]:
def time_level_window_mean_v1(input, t_range, l_range):
    output = np.zeros(shape=input.shape)

    for t in range(input.shape[0]):
        for l in range(input.shape[1]):
            for f in range(input.shape[2]):
                window = input[
                    min(max(0, t+t_range[0]), input.shape[0]):max(0, min(t+1+t_range[1], input.shape[0])),
                    min(max(0, l+l_range[0]), input.shape[1]):max(0, min(l+1+l_range[1], input.shape[1])),
                    f
                ]

                output[t,l,f] = np.mean(window) if window.size > 0 else 0.0
    
    return output

def time_level_window_mean_v2(input, t_range, l_range):
    output = np.zeros(shape=input.shape)

    for t in range(input.shape[0]):
        mint = min(max(0, t+t_range[0]), input.shape[0])
        maxt = max(0, min(t+1+t_range[1], input.shape[0]))
        
        if mint == maxt:
            continue
        
        for l in range(input.shape[1]):
            minl = min(max(0, l+l_range[0]), input.shape[1])
            maxl = max(0, min(l+1+l_range[1], input.shape[1]))
            
            if minl == maxl:
                continue
                
            output[t,l,:] = np.mean(input[mint:maxt,minl:maxl,:], axis=(0,1))
    
    return output

def time_level_window_mean_v3(input, t_range, l_range):
    min_t = min(t_range[0], 0)
    max_t = max(0, t_range[1])
    abs_t = max(abs(min_t), abs(max_t))
    
    min_l = min(l_range[0], 0)
    max_l = max(0, l_range[1])
    abs_l = max(abs(min_l), abs(max_l))
    
    kernel = np.zeros(shape=(abs_t*2 + 1, abs_l*2 + 1, 1))
    kernel[t_range[0]+abs_t:t_range[1]+abs_t+1,l_range[0]+abs_l:l_range[1]+abs_l+1,:] = 1.0
    kernel = kernel[::-1,::-1]
    
    quot = sp.ndimage.convolve(np.ones_like(input), kernel, mode='constant', cval=0.0)
    
    result = np.zeros_like(input)
    
    np.divide(
        sp.ndimage.convolve(input, kernel, mode='constant', cval=0.0),
        quot, out=result, where=quot > 0,
    )
    
    return result

In [ ]:
def get_raw_features_for_dataset(ds: TrajectoryDatasets):
    bio_features = get_bio_emissions_features(ds)
    aer_features = get_aer_emissions_features(ds) * 1e21
    ant_features = get_ant_emissions_features(ds)
    met_features = get_meteorology_features(ds)
    
    return pd.concat([
        bio_features, aer_features, ant_features, met_features,
    ], axis="columns")

In [ ]:
def get_features_from_raw_features(raw_features):
    raw_features_np = df_to_numpy(raw_features)
    
    features_np = np.concatenate([
        raw_features.index.get_level_values(0).to_numpy().reshape(
            (raw_features.index.levels[0].size, raw_features.index.levels[1].size, 1)
        ),
        raw_features.index.get_level_values(1).to_numpy().reshape(
            (raw_features.index.levels[0].size, raw_features.index.levels[1].size, 1)
        )
    ] + joblib.Parallel(n_jobs=-1)([
        joblib.delayed(time_level_window_mean_v2)(raw_features_np, t, l) for t, l in generate_time_level_windows()
    ]), axis=2)
    
    # Trim off the first two days, for which the time features are ill-defined
    features_np_trimmed = features_np[95:-1,:,:]
    
    feature_names = ["time", "level"] + generate_windowed_feature_names(raw_features.columns)
    
    features = pd.DataFrame(features_np_trimmed.reshape(
        features_np_trimmed.shape[0]*features_np_trimmed.shape[1], features_np_trimmed.shape[2],
    ), columns=feature_names).set_index(["time", "level"])
    
    return features

In [ ]:
def get_labels_for_dataset(ds: TrajectoryDatasets):
    ccn_concentration = get_ccn_concentration(ds)
    
    ccn_concentration_np = df_to_numpy(ccn_concentration)
    
    labels_np = np.concatenate([
        ccn_concentration.index.get_level_values(0).to_numpy().reshape(
            (ccn_concentration.index.levels[0].size, ccn_concentration.index.levels[1].size, 1)
        ),
        ccn_concentration.index.get_level_values(1).to_numpy().reshape(
            (ccn_concentration.index.levels[0].size, ccn_concentration.index.levels[1].size, 1)
        ),
        ccn_concentration_np.reshape(
            (ccn_concentration_np.shape[0], ccn_concentration_np.shape[1], 1)
        ),
    ], axis=2)
    
    # Trim off the first two days, for which the time features are ill-defined
    labels_np_trimmed = labels_np[96:,:,:]
    
    label_names = ["time", "level", "ccn"]
    
    labels = pd.DataFrame(labels_np_trimmed.reshape(
        labels_np_trimmed.shape[0]*labels_np_trimmed.shape[1], labels_np_trimmed.shape[2],
    ), columns=label_names).set_index(["time", "level"])
    
    return labels

In [ ]:
def hash_for_dt(dt):
    if not(isinstance(dt, tuple) or isinstance(dt, list)):
        dt = [dt]
    
    dt_str = '.'.join(dtt.strftime('%d.%m.%Y-%H:00%z') for dtt in dt)
    
    h = hashlib.shake_256()
    h.update(dt_str.encode('ascii'))
    
    return h

In [ ]:
"""
Clumped 0/1 sampler using a Markov Process

P(0) = p and P(1) = 1-p
clump = 0 => IID samples
clump -> 1 => highly correlated samples

"""
class Clump:
    def __init__(self, p=0.5, clump=0.0, rng=None):
        a = 1 - (1-p)*(1-clump)
        b = (1-a)*p/(1-p)
        
        self.C = np.array([[a, 1-a],[b, 1-b]])
        
        self.i = 0 if rng.random() < p else 1
    
    def sample(self, rng):
        p = self.C[self.i,0]
        u = rng.random()
        
        self.i = 0 if u < p else 1
        
        return self.i
    
    def steady(self, X):
        return np.matmul(X, self.C)

In [ ]:
def train_test_split(X, Y, test_size=0.25, random_state=None, shuffle=True, clump=0.0):
    assert len(X) == len(Y)
    assert type(X) == type(Y)
    assert test_size > 0.0
    assert test_size < 1.0
    assert random_state is not None
    assert clump >= 0.0
    assert clump < 1.0
    
    c = Clump(p=test_size, clump=clump, rng=random_state)
    
    if isinstance(X, pd.DataFrame):
        assert X.index.values.shape == Y.index.values.shape
        
        # Split only based on the first-level index instead of flattening
        n1 = len(X.index.levels[1])
        n0 = len(X) // n1
        
        C = np.array([c.sample(random_state) for _ in range(n0)])
        I_train, = np.nonzero(C)
        I_train = np.repeat(I_train, n1) * n1 + np.tile(np.arange(n1), len(I_train))
        I_test, = np.nonzero(1-C)
        I_test = np.repeat(I_test, n1) * n1 + np.tile(np.arange(n1), len(I_test))
    else:
        C = np.array([c.sample(random_state) for _ in range(len(X))])
        I_train, = np.nonzero(C)
        I_test, = np.nonzero(1-C)
    
    if shuffle:
        random_state.shuffle(I_train)
        random_state.shuffle(I_test)
    
    if isinstance(X, pd.DataFrame):
        X_train = X.iloc[I_train]
        X_test = X.iloc[I_test]
        
        Y_train = Y.iloc[I_train]
        Y_test = Y.iloc[I_test]
    else:
        X_train = X[I_train]
        X_test = X[I_test]
        
        Y_train = Y[I_train]
        Y_test = Y[I_test]
    
    return X_train, X_test, Y_train, Y_test

In [ ]:
def load_and_cache_dataset(dt: datetime.datetime, clump: float, datasets: dict) -> MLDataset:
    if isinstance(dt, tuple) or isinstance(dt, list):
        dt = tuple(sorted(dt))
    
    cached = datasets.get((dt, clump))
    
    if cached is not None:
        return cached
    
    if isinstance(dt, tuple) or isinstance(dt, list):
        mls = [load_and_cache_dataset(dtt, clump, datasets) for dtt in dt]

        dp = tuple(ml.paths for ml in mls)
        X_raw = pd.concat([ml.X_raw for ml in mls], axis='index')
        Y = pd.concat([ml.Y_raw for ml in mls], axis='index')

        train_features = np.concatenate([ml.X_scaler.inverse_transform(ml.X_train) for ml in mls], axis=0)
        train_labels = np.concatenate([ml.Y_scaler.inverse_transform(ml.Y_train) for ml in mls], axis=0)
        valid_features = np.concatenate([ml.X_scaler.inverse_transform(ml.X_valid) for ml in mls], axis=0)
        valid_labels = np.concatenate([ml.Y_scaler.inverse_transform(ml.Y_valid) for ml in mls], axis=0)
        test_features = np.concatenate([ml.X_scaler.inverse_transform(ml.X_test) for ml in mls], axis=0)
        test_labels = np.concatenate([ml.Y_scaler.inverse_transform(ml.Y_test) for ml in mls], axis=0)
    else:
        dp = traj_datetimes[dt]
        ds = load_trajectory_dataset(dp)

        X_raw = get_raw_features_for_dataset(ds)

        X = get_features_from_raw_features(X_raw)
        Y = np.log10(get_labels_for_dataset(ds) + 1)

        rng = np.random.RandomState(seed=int.from_bytes(hash_for_dt(dt).digest(4), 'little'))

        train_features, test_features, train_labels, test_labels = train_test_split(
            X, Y, test_size=0.25, random_state=rng, clump=clump,
        )
        train_features, valid_features, train_labels, valid_labels = train_test_split(
            train_features, train_labels, test_size=1.0/3.0, random_state=rng, clump=clump,
        )

        # Close the NetCDF dataset
        ds.out.close()

    # Scale features to N(0,1)
    # - only fit on training data
    # - OOD inputs for constants at training time are blown up
    feature_scaler = StandardScaler().fit(train_features)
    feature_scaler.scale_[np.nonzero(feature_scaler.var_ == 0.0)] = np.nan_to_num(np.inf)

    label_scaler = StandardScaler().fit(train_labels)

    train_features = feature_scaler.transform(train_features)
    train_labels = label_scaler.transform(train_labels)
    valid_features = feature_scaler.transform(valid_features)
    valid_labels = label_scaler.transform(valid_labels)
    test_features = feature_scaler.transform(test_features)
    test_labels = label_scaler.transform(test_labels)

    dataset = MLDataset(
        date=dt, paths=dp, X_raw=X_raw, Y_raw=Y,
        X_train=train_features, X_valid=valid_features, X_test=test_features,
        Y_train=train_labels, Y_valid=valid_labels, Y_test=test_labels,
        X_scaler=feature_scaler, Y_scaler=label_scaler,
    )

    datasets[(dt, clump)] = dataset
    
    return dataset

In [ ]:
DATASETS = dict()

In [ ]:
import abc

class TrainableModel(abc.ABC):
    @abc.abstractmethod
    def fit(self, X_train, Y_train):
        return self
    
    @abc.abstractmethod
    def predict(self, X_test):
        return None
    
    def predict_uncertainty(self, X_test):
        return np.zeros_like(self.predict(X_test))
    
    @classmethod
    def name(cls):
        return cls.__name__

In [ ]:
from sklearn.ensemble import RandomForestRegressor

class RandomForestModel(TrainableModel):
    def __init__(self, *args, rng=None, **kwargs):
        self.model = RandomForestRegressor(*args, random_state=rng, **kwargs)
    
    def fit(self, X_train, Y_train):
        self.Y_shape = list(Y_train.shape[1:])
        
        self.model.fit(X_train, Y_train)
        
        return self
    
    def predict(self, X_test):
        return self.model.predict(X_test).reshape([len(X_test)] + self.Y_shape)
    
    def predict_uncertainty(self, X_test):
        predictions = [
            estimator.predict(X_test) for estimator in self.model.estimators_
        ]
        
        return np.std(np.stack(predictions, axis=0), axis=0)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

class PairwiseRandomForestModel(TrainableModel):
    def __init__(self, *args, rng=None, **kwargs):
        self.model = RandomForestRegressor(*args, random_state=rng, **kwargs)
        self.rng = rng
    
    def fit(self, X_train, Y_train):
        self.Y_shape = list(Y_train.shape[1:])
        
        self.X = X_train
        self.Y = Y_train
        
        I1 = self.rng.choice(len(self.X), size=len(self.X))
        I2 = self.rng.choice(len(self.X), size=len(self.X))
        
        self.model.fit(
            np.concatenate([self.X[I1], self.X[I2]-self.X[I1]], axis=1),
            self.Y[I2]-self.Y[I1],
        )
        
        return self
    
    def predict(self, X_test):
        predictions = []
        
        for estimator in self.model.estimators_:
            I2 = self.rng.choice(len(self.X), size=len(X_test))
            
            predictions.append(self.Y[I2] - estimator.predict(
                np.concatenate([X_test, self.X[I2]-X_test], axis=1),
            ).reshape([len(X_test)] + self.Y_shape))
        
        return np.mean(np.stack(predictions, axis=0), axis=0)
    
    def predict_uncertainty(self, X_test):
        predictions = []
        
        for estimator in self.model.estimators_:
            I2 = self.rng.choice(len(self.X), size=len(X_test))
            
            predictions.append(self.Y[I2] - estimator.predict(
                np.concatenate([X_test, self.X[I2]-X_test], axis=1),
            ).reshape([len(X_test)] + self.Y_shape))
        
        return np.std(np.stack(predictions, axis=0), axis=0)

In [ ]:
class PairwiseRandomForestGaussianResidualModel(TrainableModel):
    def __init__(self, *args, rng=None, **kwargs):
        self.base_model = RandomForestRegressor(*args, random_state=rng, **kwargs)
        self.error_model = GaussianProcessRegressor(kernel=(
            RBF() + WhiteKernel()
        ), random_state=rng)
        self.rng = rng
    
    def fit(self, X_train, Y_train):
        self.Y_shape = list(Y_train.shape[1:])
        
        I_train = self.rng.choice(len(X_train), size=len(X_train)//2, replace=False)
        I_valid = np.ones(len(X_train))
        I_valid[I_train] = 0
        I_valid, = np.nonzero(I_valid)
        
        self.X = X_train[I_train]
        self.Y = Y_train[I_train]
        
        I1 = self.rng.choice(len(self.X), size=len(self.X))
        I2 = self.rng.choice(len(self.X), size=len(self.X))
        
        self.base_model.fit(
            np.concatenate([self.X[I1], self.X[I2]-self.X[I1]], axis=1),
            self.Y[I2]-self.Y[I1],
        )
        
        predictions = []
        for estimator in self.base_model.estimators_:
            I3 = self.rng.choice(len(self.X), size=len(I_valid))
            
            predictions.append(self.Y[I3] - estimator.predict(
                np.concatenate([X_train[I_valid], self.X[I3]-X_train[I_valid]], axis=1),
            ).reshape([len(I_valid)] + self.Y_shape))
        Y_pred = np.mean(np.stack(predictions, axis=0), axis=0)
        Y_range = np.std(np.stack(predictions, axis=0), axis=0)
        
        C_train = np.concatenate([X_train[I_valid], Y_pred, Y_range], axis=1)
        E_train = Y_train[I_valid] - Y_pred
        
        print(C_train.shape, E_train.shape, X_train[I_valid].shape)
        
        num_C = len(C_train)
        
        # Safety hatch to limit computing time
        if num_C < 10_000:
            I = np.arange(num_C)
        else:
            I = self.rng.choice(num_C, size=num_C//6)
        
        self.error_model.fit(C_train[I], E_train[I])
        
        return self
    
    def predict(self, X_test):
        predictions = []
        for estimator in self.base_model.estimators_:
            I2 = self.rng.choice(len(self.X), size=len(X_test))
            
            predictions.append(self.Y[I2] - estimator.predict(
                np.concatenate([X_test, self.X[I2]-X_test], axis=1),
            ).reshape([len(X_test)] + self.Y_shape))
        Y_pred = np.mean(np.stack(predictions, axis=0), axis=0)
        Y_range = np.std(np.stack(predictions, axis=0), axis=0)
        
        C_test = np.concatenate([X_test, Y_pred, Y_range], axis=1)
        
        return Y_pred + self.error_model.predict(C_test).reshape(
            [len(X_test)] + self.Y_shape
        )
    
    def predict_uncertainty(self, X_test):
        predictions = []
        for estimator in self.base_model.estimators_:
            I2 = self.rng.choice(len(self.X), size=len(X_test))
            
            predictions.append(self.Y[I2] - estimator.predict(
                np.concatenate([X_test, self.X[I2]-X_test], axis=1),
            ).reshape([len(X_test)] + self.Y_shape))
        Y_pred = np.mean(np.stack(predictions, axis=0), axis=0)
        Y_range = np.std(np.stack(predictions, axis=0), axis=0)
        
        C_test = np.concatenate([X_test, Y_pred, Y_range], axis=1)
        
        return self.error_model.predict(C_test, return_std=True)[1]

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

class GaussianProcessModel(TrainableModel):
    def __init__(self, *args, rng=None, **kwargs):
        self.model = GaussianProcessRegressor(*args, kernel=(
            RBF() + WhiteKernel()
        ), random_state=rng, **kwargs)
        self.rng = rng
    
    def fit(self, X_train, Y_train):
        self.Y_shape = list(Y_train.shape[1:])
        
        num_X = len(X_train)
        
        # Safety hatch to limit computing time
        if num_X < 10_000:
            I = np.arange(num_X)
        else:
            I = self.rng.choice(num_X, size=num_X//6)
        
        self.model.fit(X_train[I], Y_train[I])
        
        return self
    
    def predict(self, X_test):
        return self.model.predict(X_test).reshape([len(X_test)] + self.Y_shape)
    
    def predict_uncertainty(self, X_test):
        return self.model.predict(X_test, return_std=True)[1]

In [ ]:
import tensorflow as tf

def beta_nll_loss(y_mse, sigma2_pred):
    raise Exception("refit of saved model")

tf.keras.utils.get_custom_objects()["beta_nll_loss"] = beta_nll_loss

class BetaVarianceAttenuationModel(TrainableModel):
    def __init__(self, *args, beta=0.5, rng=None, **kwargs):
        self.beta = beta
        self.rng = rng
    
    def fit(self, X_train, Y_train):
        I_train = self.rng.choice(len(X_train), size=len(X_train)//2, replace=False)
        I_valid = np.ones(len(X_train))
        I_valid[I_train] = 0
        I_valid, = np.nonzero(I_valid)
        
        tf.random.set_seed(int.from_bytes(self.rng.bytes(4), "little"))
        
        self.mu_model = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=X_train.shape[1:]),
                tf.keras.layers.Dense(32, activation="relu", name="mu-1"),
                tf.keras.layers.Dense(32, activation="relu", name="mu-2"),
                tf.keras.layers.Dense(32, activation="relu", name="mu-3"),
                tf.keras.layers.Dense(Y_train.shape[1], name="mu"),
            ]
        )
        self.mu_model.compile(optimizer='adam', loss="mse", metrics=["mse", "mae"])
        
        self.mu_model.fit(
            X_train[I_train], Y_train[I_train],
            validation_data=(X_train[I_valid], Y_train[I_valid]),
            batch_size=200, epochs=200, verbose=1, callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=10,
                    restore_best_weights=True,
                )
            ],
        )
        
        Y_mse = (self.mu_model.predict(X_train).reshape(Y_train.shape) - Y_train) ** 2
        
        def beta_nll_loss(y_mse, sigma2_pred):
            return tf.stop_gradient(tf.math.pow(sigma2_pred, self.beta)) * (
                tf.math.log(sigma2_pred)*0.5 + y_mse*0.5/sigma2_pred
            )
        
        self.sigma2_model = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=X_train.shape[1:]),
                tf.keras.layers.Dense(32, activation="relu", name="sigma2-1"),
                tf.keras.layers.Dense(32, activation="relu", name="sigma2-2"),
                tf.keras.layers.Dense(32, activation="relu", name="sigma2-3"),
                tf.keras.layers.Dense(Y_train.shape[1], activation="exponential", name="sigma2"),
            ]
        )
        self.sigma2_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001), loss=beta_nll_loss,
        )
        
        self.sigma2_model.fit(
            X_train[I_valid], Y_mse[I_valid], validation_split=0.25, 
            sample_weight=Y_mse[I_valid].flatten() + 1,
            batch_size=200, epochs=200, verbose=1, callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=25,
                    restore_best_weights=True,
                )
            ],
        )
        
        return self
    
    def predict(self, X_test):
        return self.mu_model.predict(X_test)
    
    def predict_uncertainty(self, X_test):
        return np.sqrt(self.sigma2_model.predict(X_test))

In [ ]:
class MonteCarloDropout(tf.keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

class MonteCarloDropoutModel(TrainableModel):
    def __init__(self, *args, rate=0.1, samples=10, rng=None, **kwargs):
        self.rate = rate
        self.samples = samples
        self.rng = rng
    
    def fit(self, X_train, Y_train):
        I_train = self.rng.choice(len(X_train), size=len(X_train)//2, replace=False)
        I_valid = np.ones(len(X_train))
        I_valid[I_train] = 0
        I_valid, = np.nonzero(I_valid)
        
        tf.random.set_seed(int.from_bytes(self.rng.bytes(4), "little"))
        
        self.model = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=X_train.shape[1:]),
                tf.keras.layers.Dense(64, activation="relu", name="dense-1"),
                MonteCarloDropout(rate=self.rate, name="dropout-1"),
                tf.keras.layers.Dense(64, activation="relu", name="dense-2"),
                MonteCarloDropout(rate=self.rate, name="dropout-2"),
                tf.keras.layers.Dense(64, activation="relu", name="dense-3"),
                MonteCarloDropout(rate=self.rate, name="dropout-3"),
                tf.keras.layers.Dense(Y_train.shape[1], name="output"),
            ]
        )
        self.model.compile(optimizer='adam', loss="mse", metrics=["mse", "mae"])
        
        self.model.fit(
            X_train[I_train], Y_train[I_train],
            validation_data=(X_train[I_valid], Y_train[I_valid]),
            batch_size=200, epochs=200, verbose=1, callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=10,
                    restore_best_weights=True,
                )
            ],
        )
        
        return self
    
    def predict(self, X_test):
        predictions = []
        
        for _ in range(self.samples):
            predictions.append(self.model.predict(X_test))
            
        return np.mean(np.stack(predictions, axis=0), axis=0)
    
    def predict_uncertainty(self, X_test):
        predictions = []
        
        for _ in range(self.samples):
            predictions.append(self.model.predict(X_test))
            
        return np.std(np.stack(predictions, axis=0), axis=0)

In [ ]:
import tensorflow_probability as tfp

class NormalLossModel(tf.keras.Model):
    def __init__(self, indim, outdim):
        input = tf.keras.Input(shape=[indim])
        
        mu_1 = tf.keras.layers.Dense(32, activation="relu", name="mu-1")(input)
        mu_2 = tf.keras.layers.Dense(32, activation="relu", name="mu-2")(mu_1)
        mu_3 = tf.keras.layers.Dense(32, activation="relu", name="mu-3")(mu_2)
        
        mu = tf.keras.layers.Dense(outdim, name="mu")(mu_3)
        
        sigma_1 = tf.keras.layers.Dense(32, activation="relu", name="sigma-1")(input)
        sigma_2 = tf.keras.layers.Dense(32, activation="relu", name="sigma-2")(sigma_1)
        sigma_3 = tf.keras.layers.Dense(32, activation="relu", name="sigma-3")(sigma_2)
        
        sigma = tf.keras.layers.Dense(outdim, activation="softplus", name="sigma")(sigma_3)
        
        super().__init__(input, [mu, sigma])
    
    def train_step(self, data):
        x, y = data
        
        with tf.GradientTape() as tape:
            mu, sigma = self(x, training=True)
            
            uq = tfp.distributions.Normal(loc=mu, scale=sigma)
            
            loss = self.compiled_loss(y, -uq.log_prob(y))
    
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        
        self.compiled_metrics.update_state(y, mu)
        
        return {m.name: m.result() for m in self.metrics}
    
    def test_step(self, data):
        x, y = data
        
        mu, sigma = self(x, training=False)
            
        uq = tfp.distributions.Normal(loc=mu, scale=sigma)
            
        loss = self.compiled_loss(y, -uq.log_prob(y))
        
        self.compiled_metrics.update_state(y, mu)
        
        return {m.name: m.result() for m in self.metrics}

def neg_log_prob_loss(y, loss):
    return loss

tf.keras.utils.get_custom_objects()["neg_log_prob_loss"] = neg_log_prob_loss
    
class NormalUncertaintyModel(TrainableModel):
    def __init__(self, *args, rng=None, **kwargs):
        self.rng = rng
    
    def fit(self, X_train, Y_train):
        I_train = self.rng.choice(len(X_train), size=len(X_train)//2, replace=False)
        I_valid = np.ones(len(X_train))
        I_valid[I_train] = 0
        I_valid, = np.nonzero(I_valid)
        
        tf.random.set_seed(int.from_bytes(self.rng.bytes(4), "little"))
        
        self.model = NormalLossModel(X_train.shape[1], Y_train.shape[1])
        self.model.compile(optimizer='adam', loss=neg_log_prob_loss)
        
        self.model.fit(
            X_train[I_train], Y_train[I_train],
            validation_data=(X_train[I_valid], Y_train[I_valid]),
            batch_size=200, epochs=200, verbose=1, callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=15,
                    restore_best_weights=True,
                )
            ],
        )
        
        return self
    
    def predict(self, X_test):
        mu, sigma = self.model(X_test)
        
        return mu.numpy()
    
    def predict_uncertainty(self, X_test):
        mu, sigma = self.model(X_test)
        
        return sigma.numpy()

In [ ]:
import gpflow

class ResidualInputOutputModel(TrainableModel):
    def __init__(self, *args, M=250, rng=None, **kwargs):
        self.M = M
        self.rng = rng
    
    def fit(self, X_train, Y_train):
        I_train = self.rng.choice(len(X_train), size=len(X_train)//2, replace=False)
        I_valid = np.ones(len(X_train))
        I_valid[I_train] = 0
        I_valid, = np.nonzero(I_valid)
        
        tf.random.set_seed(int.from_bytes(self.rng.bytes(4), "little"))
        
        self.base_model = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=X_train.shape[1:]),
                tf.keras.layers.Dense(32, activation="relu", name="base-1"),
                tf.keras.layers.Dense(32, activation="relu", name="base-2"),
                tf.keras.layers.Dense(32, activation="relu", name="base-3"),
                tf.keras.layers.Dense(Y_train.shape[1], name="base"),
            ]
        )
        self.base_model.compile(optimizer='adam', loss="mse", metrics=["mse", "mae"])
        
        self.base_model.fit(
            X_train[I_train], Y_train[I_train],
            validation_data=(X_train[I_valid], Y_train[I_valid]),
            batch_size=200, epochs=200, verbose=1, callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=10,
                    restore_best_weights=True,
                )
            ],
        )
        
        Y_pred = self.base_model.predict(X_train[I_valid]).astype("float64")
        Y_error = (Y_train[I_valid] - Y_pred).astype("float64")
        
        # Code inspired by https://github.com/cognizant-ai-labs/rio-paper
        X_combined = np.concatenate([X_train[I_valid], Y_pred], axis=1)
        
        inducing_variable = X_combined[:self.M, :].copy()
        kernel = gpflow.kernels.SquaredExponential(
            active_dims=range(0, X_train.shape[1]),
        ) + gpflow.kernels.SquaredExponential(
            active_dims=range(X_train.shape[1], X_train.shape[1] + Y_train.shape[1]),
        )
        
        self.rio_model = gpflow.models.SVGP(
            kernel=kernel, likelihood=gpflow.likelihoods.Gaussian(),
            inducing_variable=inducing_variable,
        )
        training_loss = self.rio_model.training_loss_closure((X_combined, Y_error))
        
        optimizer = gpflow.optimizers.Scipy()
        optimizer.minimize(training_loss, variables=self.rio_model.trainable_variables)
        
        return self
    
    def predict(self, X_test):
        Y_pred = self.base_model.predict(X_test)
        
        X_combined = np.concatenate([X_test, Y_pred], axis=1)
        
        mean, var = self.rio_model.predict_f(X_combined)
        
        return Y_pred + mean.numpy()
    
    def predict_uncertainty(self, X_test):
        Y_pred = self.base_model.predict(X_test)
        
        X_combined = np.concatenate([X_test, Y_pred], axis=1)
        
        mean, var = self.rio_model.predict_f(X_combined)
        
        return np.sqrt(var.numpy())

In [ ]:
import tensorflow as tf

class ResidualRegressionModel(TrainableModel):
    def __init__(self, *args, rng=None, **kwargs):
        self.rng = rng
    
    def fit(self, X_train, Y_train):
        I_train = self.rng.choice(len(X_train), size=len(X_train)//2, replace=False)
        I_valid = np.ones(len(X_train))
        I_valid[I_train] = 0
        I_valid, = np.nonzero(I_valid)
        
        tf.random.set_seed(int.from_bytes(self.rng.bytes(4), "little"))
        
        self.base_model = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=X_train.shape[1:]),
                tf.keras.layers.Dense(32, activation="relu", name="base-1"),
                tf.keras.layers.Dense(32, activation="relu", name="base-2"),
                tf.keras.layers.Dense(32, activation="relu", name="base-3"),
                tf.keras.layers.Dense(Y_train.shape[1], name="base"),
            ]
        )
        self.base_model.compile(optimizer='adam', loss="mse", metrics=["mse", "mae"])
        
        self.base_model.fit(
            X_train[I_train], Y_train[I_train],
            validation_data=(X_train[I_valid], Y_train[I_valid]),
            batch_size=200, epochs=200, verbose=1, callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=10,
                    restore_best_weights=True,
                )
            ],
        )
        
        Y_error = self.base_model.predict(X_train[I_valid]).reshape(
            Y_train[I_valid].shape
        ) - Y_train[I_valid]
        
        self.error_model = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=X_train.shape[1:]),
                tf.keras.layers.Dense(32, activation="relu", name="error-1"),
                tf.keras.layers.Dense(32, activation="relu", name="error-2"),
                tf.keras.layers.Dense(32, activation="relu", name="error-3"),
                tf.keras.layers.Dense(Y_train.shape[1], name="error"),
            ]
        )
        self.error_model.compile(optimizer='adam', loss="mse", metrics=["mse", "mae"])
        
        self.error_model.fit(
            X_train[I_valid], Y_error, validation_split=0.25, 
            sample_weight=(Y_error.flatten() ** 2) + 1,
            batch_size=200, epochs=200, verbose=1, callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=25,
                    restore_best_weights=True,
                )
            ],
        )
        
        return self
    
    def predict(self, X_test):
        return self.base_model.predict(X_test)
    
    def predict_uncertainty(self, X_test):
        return np.abs(self.error_model.predict(X_test))

In [ ]:
def lb_loss(y_true, lb_pred):
    loss = y_true - lb_pred
    return loss * (0.159 + tf.math.minimum(tf.math.sign(loss), 0.0))

def mb_loss(y_true, lb_pred):
    loss = y_true - lb_pred
    return loss * (0.5 + tf.math.minimum(tf.math.sign(loss), 0.0))

def ub_loss(y_true, lb_pred):
    loss = y_true - lb_pred
    return loss * (0.841 + tf.math.minimum(tf.math.sign(loss), 0.0))

tf.keras.utils.get_custom_objects()["lb_loss"] = lb_loss
tf.keras.utils.get_custom_objects()["mb_loss"] = mb_loss
tf.keras.utils.get_custom_objects()["ub_loss"] = ub_loss

class QuantileRegressionModel(TrainableModel):
    def __init__(self, *args, rng=None, **kwargs):
        self.rng = rng
    
    def fit(self, X_train, Y_train):
        I_train = self.rng.choice(len(X_train), size=len(X_train)//2, replace=False)
        I_valid = np.ones(len(X_train))
        I_valid[I_train] = 0
        I_valid, = np.nonzero(I_valid)
        
        tf.random.set_seed(int.from_bytes(self.rng.bytes(4), "little"))
        
        self.lb_model = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=X_train.shape[1:]),
                tf.keras.layers.Dense(32, activation="relu", name="lb-1"),
                tf.keras.layers.Dense(32, activation="relu", name="lb-2"),
                tf.keras.layers.Dense(32, activation="relu", name="lb-3"),
                tf.keras.layers.Dense(Y_train.shape[1], name="lb"),
            ]
        )
        self.lb_model.compile(optimizer='adam', loss=lb_loss)
        
        self.lb_model.fit(
            X_train[I_train], Y_train[I_train],
            validation_data=(X_train[I_valid], Y_train[I_valid]),
            batch_size=200, epochs=200, verbose=1, callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=10,
                    restore_best_weights=True,
                )
            ],
        )
        
        self.mb_model = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=X_train.shape[1:]),
                tf.keras.layers.Dense(32, activation="relu", name="mb-1"),
                tf.keras.layers.Dense(32, activation="relu", name="mb-2"),
                tf.keras.layers.Dense(32, activation="relu", name="nb-3"),
                tf.keras.layers.Dense(Y_train.shape[1], name="mb"),
            ]
        )
        self.mb_model.compile(optimizer='adam', loss=mb_loss, metrics=["mse", "mae"])
        
        self.mb_model.fit(
            X_train[I_train], Y_train[I_train],
            validation_data=(X_train[I_valid], Y_train[I_valid]),
            batch_size=200, epochs=200, verbose=1, callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=10,
                    restore_best_weights=True,
                )
            ],
        )
        
        self.ub_model = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=X_train.shape[1:]),
                tf.keras.layers.Dense(32, activation="relu", name="ub-1"),
                tf.keras.layers.Dense(32, activation="relu", name="ub-2"),
                tf.keras.layers.Dense(32, activation="relu", name="ub-3"),
                tf.keras.layers.Dense(Y_train.shape[1], name="ub"),
            ]
        )
        self.ub_model.compile(optimizer='adam', loss=ub_loss)
        
        self.ub_model.fit(
            X_train[I_train], Y_train[I_train],
            validation_data=(X_train[I_valid], Y_train[I_valid]),
            batch_size=200, epochs=200, verbose=1, callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=10,
                    restore_best_weights=True,
                )
            ],
        )
        
        return self
    
    def predict(self, X_test):
        return self.mb_model.predict(X_test)
    
    def predict_uncertainty(self, X_test):
        return np.abs(
            self.ub_model.predict(X_test) - self.lb_model.predict(X_test)
        ) * 0.5

In [ ]:
class IntervalBoundsModel(tf.keras.Model):
    def __init__(self, indim, outdim):
        input = tf.keras.Input(shape=[indim])
        
        lb_1 = tf.keras.layers.Dense(32, activation="relu", name="lb-1")(input)
        lb_2 = tf.keras.layers.Dense(32, activation="relu", name="lb-2")(lb_1)
        lb_3 = tf.keras.layers.Dense(32, activation="relu", name="lb-3")(lb_2)
        
        lb = tf.keras.layers.Dense(outdim, name="lb")(lb_3)
        
        ub_1 = tf.keras.layers.Dense(32, activation="relu", name="ub-1")(input)
        ub_2 = tf.keras.layers.Dense(32, activation="relu", name="ub-2")(ub_1)
        ub_3 = tf.keras.layers.Dense(32, activation="relu", name="ub-3")(ub_2)
        
        ub = tf.keras.layers.Dense(outdim, name="ub")(ub_3)
        
        super().__init__(input, [lb, ub])
        
        self.k = self.add_weight(initializer="ones", trainable=False, name="k")
    
    def train_step(self, data):
        x, y = data
        
        with tf.GradientTape() as tape:
            p_lb, p_ub = 0.159, 0.841
            y_lb, y_ub = self(x, training=True)
            
            kI = tf.math.abs((y_ub + y_lb - y * 2.0) / (y_ub - y_lb))
            kI = tf.where(tf.math.is_nan(kI), tf.ones_like(kI), kI)
            pl = tfp.stats.percentile(kI, ((p_ub-p_lb) - 0.02) * 100, interpolation="lower")
            pu = tfp.stats.percentile(kI, ((p_ub-p_lb) + 0.02) * 100, interpolation="higher")
            cI = tf.cast((kI >= pl) & (kI <= pu), y.dtype)
            kB = tf.reduce_sum(kI * cI) / tf.reduce_sum(cI)
            
            y_mb = (y_ub + y_lb) * 0.5
            
            loss = self.compiled_loss(y, (y - y_mb)**2 + kB * tf.math.abs(y_ub - y_lb))
    
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        
        self.compiled_metrics.update_state(y, y_mb)
        
        return {m.name: m.result() for m in self.metrics}
    
    def test_step(self, data):
        x, y = data
        
        p_lb, p_ub = 0.159, 0.841
        y_lb, y_ub = self(x, training=True)

        kI = tf.math.abs((y_ub + y_lb - y * 2.0) / (y_ub - y_lb))
        kI = tf.where(tf.math.is_nan(kI), tf.ones_like(kI), kI)
        
        self.k.assign(
            tfp.stats.percentile(kI, (p_ub-p_lb) * 100, interpolation="linear")
        )
        
        y_mb = (y_ub + y_lb) * 0.5

        loss = self.compiled_loss(y, (y - y_mb)**2 + self.k * tf.math.abs(y_ub - y_lb))
        
        self.compiled_metrics.update_state(y, y_mb)
        
        return {m.name: m.result() for m in self.metrics}

def interval_loss(y, loss):
    return loss

tf.keras.utils.get_custom_objects()["interval_loss"] = interval_loss
    
class IntervalRegressionModel(TrainableModel):
    def __init__(self, *args, rng=None, **kwargs):
        self.rng = rng
    
    def fit(self, X_train, Y_train):
        I_train = self.rng.choice(len(X_train), size=len(X_train)//2, replace=False)
        I_valid = np.ones(len(X_train))
        I_valid[I_train] = 0
        I_valid, = np.nonzero(I_valid)
        
        tf.random.set_seed(int.from_bytes(self.rng.bytes(4), "little"))
        
        self.model = IntervalBoundsModel(X_train.shape[1], Y_train.shape[1])
        self.model.compile(optimizer='adam', loss=interval_loss, metrics=["mse", "mae"])
        
        self.model.fit(
            X_train[I_train], Y_train[I_train],
            validation_data=(X_train[I_valid], Y_train[I_valid]),
            batch_size=200, epochs=200, verbose=1, callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=15,
                    restore_best_weights=True,
                )
            ],
        )
        
        return self
    
    def predict(self, X_test):
        lb, ub = self.model(X_test)
        
        return (lb.numpy() + ub.numpy()) * 0.5
    
    def predict_uncertainty(self, X_test):
        lb, ub = self.model(X_test)
        
        return np.abs((ub.numpy() - lb.numpy()) * self.model.k)

In [ ]:
def train_and_cache_model(dt: datetime.datetime, clump: float, datasets: dict, models: dict, cls, *args, **kwargs):
    if isinstance(dt, tuple) or isinstance(dt, list):
        dt = tuple(sorted(dt))
    
    model_key = (cls.__name__, dt, clump)
    
    cached = models.get(model_key)
    
    if cached is not None:
        return cached
    
    model_path = f"{cls.__name__.lower()}.uq.{hash_for_dt(dt).hexdigest(8)}.{clump}.jl"
    
    if Path(model_path).exists():    
        model = joblib.load(model_path)
        
        models[model_key] = model
        
        return model
    
    dataset = load_and_cache_dataset(dt, clump, datasets)
    
    rng = np.random.RandomState(seed=int.from_bytes(hash_for_dt(dt).digest(4), 'little'))
    
    model = cls(*args, rng=rng, **kwargs).fit(
        X_train=dataset.X_train, Y_train=dataset.Y_train,
    )
    
    joblib.dump(model, model_path)
    
    models[model_key] = model
    
    return model

In [ ]:
dt_train = [
    datetime.datetime(year=2018, month=5, day=14, hour=10),
    datetime.datetime(year=2018, month=5, day=15, hour=19),
    datetime.datetime(year=2018, month=5, day=17, hour=0),
    datetime.datetime(year=2018, month=5, day=19, hour=4),
    datetime.datetime(year=2018, month=5, day=21, hour=15),
    datetime.datetime(year=2018, month=5, day=23, hour=13),
]

dt_test = [
    dt + datetime.timedelta(hours=dh) for dt in dt_train for dh in [-4, -2, -1, 0, 1, 2, 4]
]

ds_train = load_and_cache_dataset(dt_train, 0.75, DATASETS)
ds_test = load_and_cache_dataset(dt_test, 0.75, DATASETS)

In [ ]:
class OptimalUncertaintyQuantifierModel(TrainableModel):
    def __init__(self, *args, rng=None, **kwargs):
        self.rng = rng
    
    def fit(self, X_train, Y_train):
        X_train = ds_train.X_train
        Y_train = ds_train.Y_train
        X_valid = ds_train.X_valid
        Y_valid = ds_train.Y_valid
        X_test = ds_train.X_scaler.transform(ds_test.X_scaler.inverse_transform(ds_test.X_test))
        Y_test = ds_train.Y_scaler.transform(ds_test.Y_scaler.inverse_transform(ds_test.Y_test))
        
        self.xTrain = X_train[0]
        self.xValid = X_valid[0]
        self.xTest = X_test[0]
        
        self.Y_train = Y_train
        self.Y_valid = Y_valid
        self.Y_test = Y_test
        
        self.E_train = np.abs(self.rng.normal(
            loc=0.0, scale=0.2, size=len(Y_train),
        ))
        self.E_valid = np.abs(self.rng.normal(
            loc=0.0, scale=0.2, size=len(Y_valid),
        ))
        self.E_test = np.abs(self.rng.normal(
            loc=0.0, scale=0.2, size=len(Y_test),
        ))
        
        return self
    
    def predict(self, X_test):
        if (X_test[0] == self.xTrain).all():
            Y = self.Y_train
            E = self.E_train
        elif (X_test[0] == self.xValid).all():
            Y = self.Y_valid
            E = self.E_valid
        elif (X_test[0] == self.xTest).all():
            Y = self.Y_test
            E = self.E_test
        else:
            assert False
        
        return self.rng.normal(loc=Y, scale=E.reshape(Y.shape))
    
    def predict_uncertainty(self, X_test):
        if (X_test[0] == self.xTrain).all():
            E = self.E_train
        elif (X_test[0] == self.xValid).all():
            E = self.E_valid
        elif (X_test[0] == self.xTest).all():
            E = self.E_test
        else:
            assert False
        
        return E
    
    @classmethod
    def name(cls):
        return "Ideal Uncertainty Quantifier Reference"

In [ ]:
Y_true = ds_train.Y_scaler.inverse_transform(ds_train.Y_test)
Y_pred = np.random.normal(loc=Y_true, scale=1.0)
Y_stdv = np.abs(np.random.normal(loc=1.0, scale=0.1, size=Y_true.shape))
Y_err = Y_true - Y_pred

ps = []
us = []
os = []

N = 1000

for i in range(N+1):
    ps.append(i/N)

    Yp = Y_pred.flatten() + np.nan_to_num(Y_stdv.flatten()*2.0*sp.stats.norm.ppf(i/N), nan=0.0)
    us.append(np.mean(Y_true.flatten() <= Yp))
    
    Yp = Y_pred.flatten() + np.nan_to_num(Y_stdv.flatten()*0.5*sp.stats.norm.ppf(i/N), nan=0.0)
    os.append(np.mean(Y_true.flatten() <= Yp))

fig, ax = plt.subplots(1, 1, figsize=(4, 4))
    
ax.set_xlabel("expected cdf proportion $p$")
ax.set_ylabel("observed cdf proportion")

ax.set_xlim((0.0, 1.0))
ax.set_ylim((0.0, 1.0))

ax.fill_between([0, 0.5], [0, 0], [0, 0.5], facecolor="none", edgecolor="#d0d0d0", hatch="\\\\")
ax.fill_between([0.5, 1], [0.5, 1], [1, 1], facecolor="none", edgecolor="#d0d0d0", hatch="\\\\")

lb = sp.stats.norm.cdf(-1.0)
ax.axvline(lb, 0.0, lb, c="black", ls=":")
ax.axhline(lb, 0.0, lb, c="black", ls=":")

ax.axvline(0.5, 0.0, 0.5, c="black", ls=":")
ax.axhline(0.5, 0.0, 0.5, c="black", ls=":")

ub = sp.stats.norm.cdf(1.0)
ax.axvline(ub, 0.0, ub, c="black", ls=":")
ax.axhline(ub, 0.0, ub, c="black", ls=":")

ax.set_xticks([lb, 0.5, ub])
ax.set_xticklabels([fr"${lb*100:.3}\%$", r"$50\%$", fr"${ub*100:.3}\%$"])

ax.set_yticks([lb, 0.5, ub])
ax.set_yticklabels([
    r"P$(Y \leq (\hat{\mu} - \hat{\sigma}))$",
    r"P$(Y \leq \hat{\mu})$",
    r"P$(Y \leq (\hat{\mu} + \hat{\sigma}))$",
], rotation=90, va="center")

plt.plot([0.0, 1.0], [0.0, 1.0], c="black", ls="-", label="ideal quantifier")
plt.plot(ps, us, c="#2cbdfe", ls="-.", label="underconfident")
plt.plot(ps, os, c="#f5b14c", ls="--", label="overconfident")
plt.plot([0.0, 1.0], [0.5, 0.5], c="#ff0000", ls=(0, (5, 5)), label="zero uncertainty")
ax.legend(loc="lower right")

ax.set_title(r"Examples of Statistical Consistency")

plt.savefig(f"uq.consistency-examples.pdf", dpi=100, transparent=True, bbox_inches='tight')
# plt.show()
plt.close(fig)

In [ ]:
from sklearn.isotonic import IsotonicRegression

MODELS = dict()

N = 1000

for cls, args, kwargs in [
    (OptimalUncertaintyQuantifierModel, [], dict()),
    (RandomForestModel, [], dict(n_estimators=10, min_samples_leaf=5, max_features=1.0/3.0)),
    (PairwiseRandomForestModel, [], dict(n_estimators=10, min_samples_leaf=5, max_features=1.0/3.0)),
    (ResidualRegressionModel, [], dict()),
    (QuantileRegressionModel, [], dict()),
    (IntervalRegressionModel, [], dict()),
    (BetaVarianceAttenuationModel, [], dict()),
    (MonteCarloDropoutModel, [], dict()),
    (NormalUncertaintyModel, [], dict()),
    (GaussianProcessModel, [], dict()),
    (PairwiseRandomForestGaussianResidualModel, [], dict(n_estimators=10, max_features="sqrt")),
    (ResidualInputOutputModel, [], dict()),
]:      
    model = train_and_cache_model(dt_train, 0.75, DATASETS, MODELS, cls, *args, **kwargs)
    
    Y_true_valid = ds_train.Y_scaler.inverse_transform(ds_train.Y_valid)
    Y_pred_valid = ds_train.Y_scaler.inverse_transform(
        model.predict(ds_train.X_valid).reshape(Y_true_valid.shape)
    )
    Y_stdv_valid = model.predict_uncertainty(ds_train.X_valid).reshape(
        Y_true_valid.shape
    ) * ds_train.Y_scaler.scale_
    
    Zc = np.sort((Y_true_valid.flatten() - Y_pred_valid.flatten()) / Y_stdv_valid.flatten())
    
    qis = sp.stats.norm(loc=Y_pred_valid[:,0], scale=Y_stdv_valid[:,0]).cdf(Y_true_valid[:,0])
    qos = []
    for p in qis:
        qos.append(np.mean(qis <= p))
    qos = np.array(qos)
    
    Q = IsotonicRegression(y_min=0.0, y_max=1.0, increasing=True, out_of_bounds="clip").fit(qis, qos)

    Y_true = ds_test.Y_scaler.inverse_transform(ds_test.Y_test)
    Y_pred = ds_train.Y_scaler.inverse_transform(model.predict(
        ds_train.X_scaler.transform(ds_test.X_scaler.inverse_transform(ds_test.X_test))
    ).reshape(Y_true.shape))
    Y_stdv = model.predict_uncertainty(
        ds_train.X_scaler.transform(ds_test.X_scaler.inverse_transform(ds_test.X_test))
    ).reshape(Y_true.shape) * ds_train.Y_scaler.scale_
    Y_err = Y_true - Y_pred
    
    up = np.linspace(0.0, 20.0, num=20*1000)
    un = np.linspace(-20.0, 0.0, num=20*1000)
    zp = np.linspace(0.0, 25*25, num=25*25*100)

    mu_new_cdf = []
    sigma_new_cdf = []

    for i in tqdm.tqdm(range(Y_pred.shape[0])):
        mnc = sp.integrate.trapezoid(1.0 - Q.predict(
            sp.stats.norm(loc=Y_pred[i][0], scale=Y_stdv[i][0]).cdf(up)
        ), dx=up[1]-up[0]) - sp.integrate.trapezoid(Q.predict(
            sp.stats.norm(loc=Y_pred[i][0], scale=Y_stdv[i][0]).cdf(un)
        ), dx=un[1]-un[0])

        snc = sp.integrate.trapezoid(
            1.0 - (
                Q.predict(
                    sp.stats.norm(loc=Y_pred[i][0], scale=Y_stdv[i][0]).cdf(np.sqrt(zp))
                ) - Q.predict(
                    sp.stats.norm(loc=Y_pred[i][0], scale=Y_stdv[i][0]).cdf(-np.sqrt(zp))
                )
            ), dx=zp[1]-zp[0],
        ) - mnc**2

        if snc <= 0.0:
            raise Exception(f"mnc={mnc} snc={snc} Y_pred={Y_pred[i][0]} Y_stdv={Y_stdv[i][0]}")

        mu_new_cdf.append(mnc)
        sigma_new_cdf.append(np.sqrt(snc))

    mu_new_cdf = np.array(mu_new_cdf)
    sigma_new_cdf = np.array(sigma_new_cdf)
    
    # Normalise the hist2d s.t. sampling frequency does not distort the picture
    bin_counts, bin_edges = np.histogram(Y_err.flatten(), bins=101)
    weights = 1.0 / np.maximum(100, bin_counts[np.searchsorted(bin_edges, Y_err.flatten()) - 1])
    
    
    fig, ax = plt.subplots(1, 1, figsize=(5, 4))

    ax.set_xlabel(r"prediction error $y - \hat{y}$")
    ax.set_ylabel(r"prediction uncertainty $\hat{\sigma}$")

    ax.hist2d(
        Y_err.flatten(), Y_stdv.flatten(), bins=101, weights=weights,
        cmin=1/101, cmap=uncertainty_cmap, rasterized=True,
    )
    cb = fig.colorbar(mpl.cm.ScalarMappable(
        norm=None, cmap=density_cmap,
    ), ax=ax, extend='min', extendrect=True, ticks=[0.0])
    cb.set_label(r"frequency $\sim$ correlation strength", labelpad=-11)

    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    dlim = max(abs(xlim[0]), ylim[1])

    ax.plot([-dlim, 0], [dlim, 0], c="black", ls=(5, (5, 5)))
    ax.plot([-dlim, 0], [dlim, 0], c="white", ls=(0, (5, 5)))

    dlim = max(xlim[1], ylim[1])

    ax.plot([0, dlim], [0, dlim], c="black", ls=(5, (5, 5)))
    ax.plot([0, dlim], [0, dlim], c="white", ls=(0, (5, 5)))

    pr = sp.stats.pearsonr(Y_stdv.flatten(), np.abs(Y_err.flatten())).statistic
    sr = sp.stats.spearmanr(Y_stdv.flatten(), np.abs(Y_err.flatten())).correlation

    ax.text(
        0.5, 0.95, fr"Pearson $\rho = {pr:.3}$" + "\n" + f"Spearman's $r_s = {sr:.3}$",
        ha='center', va='top', transform=ax.transAxes,
        bbox=dict(facecolor='white', alpha=0.75, edgecolor='black'),
    )

    ax.set_title(cls.name())

    plt.savefig(f"uq.{cls.__name__.lower()}-correlation.pdf", dpi=100, transparent=True, bbox_inches='tight')
    # plt.show()
    plt.close(fig)

    
    ps = []
    qs = []
    cs = []
    ms = []
    
    raw_err = 0.0
    crude_err = 0.0
    cdf_err = 0.0

    N = 1000

    for i in range(N+1):
        p = i/N
        ps.append(p)

        Yp = Y_pred.flatten() + Y_stdv.flatten()*sp.stats.norm.ppf(p)
        qs.append(np.mean(Y_true.flatten() <= Yp))
        raw_err += (p - np.mean(Y_true.flatten() < Yp)) ** 2
        
        mn = Y_pred.flatten() + np.mean(Zc) * Y_stdv.flatten()
        sn = Y_stdv.flatten() * np.std(Zc)
        Yp = mn + sn*sp.stats.norm.ppf(p)
        cs.append(np.mean(Y_true.flatten() <= Yp))
        crude_err += (p - np.mean(Y_true.flatten() < Yp)) ** 2

        Yp = mu_new_cdf + sigma_new_cdf*sp.stats.norm.ppf(p)
        ms.append(np.mean(Y_true.flatten() <= Yp))
        cdf_err += (p - np.mean(Y_true.flatten() < Yp)) ** 2
    
    raw_err = np.sqrt(raw_err / N)
    crude_err = np.sqrt(crude_err / N)
    cdf_err = np.sqrt(cdf_err / N)
    
    raw_sdcv = np.std(Y_stdv.flatten()) / np.mean(Y_stdv.flatten())
    # Same as raw_sdcv since sigma is just multiplied by a const
    crude_sdcv = np.std(Y_stdv.flatten() * np.std(Zc)) / np.mean(Y_stdv.flatten() * np.std(Zc))
    cdf_sdcv = np.std(sigma_new_cdf) / np.mean(sigma_new_cdf)
    
    raw_piw = np.mean(Y_stdv.flatten())
    crude_piw = np.mean(Y_stdv.flatten() * np.std(Zc))
    cdf_piw = np.mean(sigma_new_cdf)
    

    fig, ax = plt.subplots(1, 1, figsize=(4, 4))

    ax.set_xlabel("expected cdf proportion $p$")
    ax.set_ylabel("observed cdf proportion")

    ax.set_xlim((0.0, 1.0))
    ax.set_ylim((0.0, 1.0))

    ax.fill_between([0, 0.5], [0, 0], [0, 0.5], facecolor="none", edgecolor="#d0d0d0", hatch="\\\\")
    ax.fill_between([0.5, 1], [0.5, 1], [1, 1], facecolor="none", edgecolor="#d0d0d0", hatch="\\\\")

    lb = sp.stats.norm.cdf(-1.0)
    ax.axvline(lb, 0.0, qs[int(lb*N)], c="black", ls=":")
    ax.axhline(qs[int(lb*N)], 0.0, lb, c="#ff0000", ls=":")

    ax.axvline(0.5, 0.0, qs[int(0.5*N)], c="black", ls=":")
    ax.axhline(qs[int(0.5*N)], 0.0, 0.5, c="#ff0000", ls=":")

    ub = sp.stats.norm.cdf(1.0)
    ax.axvline(ub, 0.0, qs[int(ub*N)], c="black", ls=":")
    ax.axhline(qs[int(ub*N)], 0.0, ub, c="#ff0000", ls=":")

    ax.plot([0.0, 1.0], [0.0, 1.0], c="black")

    ax.text(
        0.25, 0.91, (
            "$\downarrow$ RMSCE $\downarrow$" + "\n" +
            fr"raw: ${raw_err:.2}$" + "\n" +
            fr"CRUDE: ${crude_err:.2}$" + "\n" +
            fr"CDF: ${cdf_err:.2}$"
        ),
        ha='center', va='top', transform=ax.transAxes,
        bbox=dict(facecolor='white', alpha=0.75, edgecolor='black'),
    )

    ax.set_xticks([lb, 0.5, ub])
    ax.set_xticklabels([fr"${lb*100:.3}\%$", r"$50\%$", fr"${ub*100:.3}\%$"])

    yticks = [qs[int(0.5*N)]]
    yticklabels = [r"P$(Y \leq \hat{\mu})$"]

    if ((qs[int(0.5*N)] - qs[int(lb*N)]) > 0.25) and (qs[int(lb*N)] > 0.08):
        yticks.insert(0, qs[int(lb*N)])
        yticklabels.insert(0, r"P$(Y \leq (\hat{\mu} - \hat{\sigma}))$")

    if ((qs[int(ub*N)] - qs[int(0.5*N)]) > 0.25) and (qs[int(ub*N)] < 0.92):
        yticks.append(qs[int(ub*N)])
        yticklabels.append(r"P$(Y \leq (\hat{\mu} + \hat{\sigma}))$")

    ax.set_yticks(yticks)
    ax.set_yticklabels(yticklabels, rotation=90, va="center", color="#ff0000")
    ax.tick_params(axis="y", colors="#ff0000")

    ax.plot(ps, qs, c="#ff0000", label=r"$\hat{\sigma}_{raw}$")
    ax.plot(ps, cs, c="#f5b14c", ls="--", label=r"$\hat{\sigma}_{CRUDE}$")
    ax.plot(ps, ms, c="#2cbdfe", ls="-.", label=r"$\hat{\sigma}_{CDF}$")

    ax.legend(loc="lower right")

    ax.set_title(r"Statistical Consistency of $\hat{\sigma}$")

    plt.savefig(f"uq.{cls.__name__.lower()}-consistency.pdf", dpi=100, transparent=True, bbox_inches='tight')
    # plt.show()
    plt.close(fig)
    
    
    ci_widths = [0.5, 0.95]

    fig, ax = plt.subplots(1, 1, figsize=(4, 4))

    bp = ax.boxplot(
        np.concatenate([
            s * sp.stats.norm.ppf(0.5+w*0.5) for w in ci_widths for s in [
                Y_stdv, Y_stdv * np.std(Zc), sigma_new_cdf.reshape(Y_stdv.shape),
            ]
        ], axis=1), whis=(5, 95), showfliers=False, medianprops=dict(color="black"),
    )

    for i, m in enumerate(bp["medians"]):
        m.set(color=["#ff0000", "#f5b14c", "#2cbdfe"][i%3])

    ax.set_xticks(np.arange(len(ci_widths)*3)+1, minor=False)
    ax.set_xticklabels([
        n for _ in ci_widths for n in [
            r"$\hat{\sigma}_{raw}$", r"$\hat{\sigma}_{CRUDE}$", r"$\hat{\sigma}_{CDF}$",
        ]
    ], minor=False)

    ax.set_xticks([2.0001 + i*3 for i in range(len(ci_widths))], minor=True)
    ax.set_xticklabels([fr"${int(w*100)}\%$" for w in ci_widths], minor=True)

    for tick in ax.xaxis.get_minor_ticks():
        tick.set_pad(25)


    def format_metric(x):
        if x >= 100.0:
            return int(x)

        if x >= 10.0:
            return int(x*10)/10

        return int(x*100)/100


    ax.text(
        0.2, 0.95, (
            "$\downarrow$ PIW $\downarrow$" + "\n" +
            fr"raw: ${format_metric(raw_piw)}$" + "\n"
            fr"CRUDE: ${format_metric(crude_piw)}$" + "\n"
            fr"CDF: ${format_metric(cdf_piw)}$"
        ),
        ha='center', va='top', transform=ax.transAxes,
        bbox=dict(facecolor='white', alpha=0.75, edgecolor='black'),
    )

    ax.text(
        0.2, 0.675, (
            r"$\uparrow$ SDCV $\uparrow$" + "\n" +
            fr"raw: ${format_metric(raw_sdcv)}$" + "\n" +
            fr"CRUDE: ${format_metric(crude_sdcv)}$" + "\n" +
            fr"CDF: ${format_metric(cdf_sdcv)}$"
        ),
        ha='center', va='top', transform=ax.transAxes,
        bbox=dict(facecolor='white', alpha=0.75, edgecolor='black'),
    )

    ax.set_title(r"UQ Sharpness and Dispersion")

    ax.set_xlabel("confidence interval width")
    ax.set_ylabel(r"uncertainty in $\log_{10}(CCN)$")

    plt.savefig(f"uq.{cls.__name__.lower()}-sharpness.pdf", dpi=100, transparent=True, bbox_inches='tight')
    # plt.show()
    plt.close(fig)